In [20]:
#           	Model	Accuracy	AUC	Recall	Prec.	F1	Kappa	MCC
# 0	Voting Classifier	0.7983	0.8311	0.5374	0.7953	0.6414	0.5083	0.5272

In [1]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
train_err = pd.read_csv('./preprocessed data/new_train_err.csv', parse_dates=['time'])
train = pd.read_csv('./training set/train_case1.csv')
train_problem = pd.read_csv('./preprocessed data/new_train_problem.csv', parse_dates=['time'])

## 학습 진행은 성능이 가장 좋았던 model_with_period과 같이 진행

In [5]:
train_err['day'] = train_err.time.dt.day
train_err['hour'] = train_err.time.dt.hour

In [6]:
user_day_df = train_err.groupby('user_id')['day'].unique().to_frame().reset_index()
user_day_df['period'] = 0 

for i in range(len(user_day_df.index)):
   user_day_df['period'][i] = len(user_day_df['day'][i])

In [7]:
user_day_df.index = user_day_df['user_id'] - 10000
user_day_df.drop(columns=['user_id', 'day'], inplace=True)
train = train.join(user_day_df)
train

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,fwver_03.11.1141,fwver_03.11.1149,fwver_04.16.2641,fwver_04.16.3345,fwver_04.16.3439,fwver_04.16.3569,fwver_04.16.3571,fwver_04.22.1442,fwver_04.22.1656,fwver_04.22.1666,fwver_04.22.1684,fwver_04.22.1778,fwver_04.33.1095,fwver_04.33.1125,fwver_04.33.1149,fwver_04.33.1171,fwver_04.73.2571,fwver_04.82.1730,fwver_05.15.2090,fwver_05.15.2092,fwver_05.15.2114,fwver_05.15.2120,fwver_05.15.2122,fwver_05.15.3104,fwver_05.66.3571,fwver_8.5.3,errtype_1,errtype_2,errtype_3,errtype_5,errtype_6,errtype_8,errtype_9,errtype_10,errtype_13,errtype_14,errtype_15,errtype_17,errtype_18,errtype_19,errtype_20,errtype_22,errtype_24,errtype_25,errtype_26,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,multiple_model,quality_0,quality_1,quality_5,quality_6,quality_8,quality_11,quality_12,period
0,316,0.0,0.0,0.0,316.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,0.0,0.0,7.0,1.0,10.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,30
1,2345,0.0,0.0,2345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,1.0,0.0,0.0,0.0,1.0,8.0,143.0,0.0,4.0,1.0,3.0,756.0,5.0,1.0,22.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,0.0,0.0,113.0,56.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30
2,305,0.0,0.0,0.0,305.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,0.0,0.0,1.0,1.0,4.0,52.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,-1.0,30.0,45.0,0.0,-1.0,0.0,29
3,297,0.0,0.0,297.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,45.0,1.0,0.0,0.0,0.0,28.0,0.0,0.0,57.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,2.0,0.0,17.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30
4,777,777.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,132.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,143.0,0.0,0.0,0.0,0.0,140.0,0.0,0.0,33.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,2.0,0,-1.0,-1.0,3.0,86.0,0.0,-1.0,0.0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,191,0.0,0.0,191.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,0.0,0.0,0.0,5.0,8.0,26.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,22.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0,0,-2.0,-2.0,3.0,-2.0,0.0,-2.0,0.0,10
14996,4,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
14997,818,818.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,358.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,1.0,0.0,0.0,0.0,3.0,1.0,181.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,48.0,0.0,223.0,0.0,16.0,12.0,0.0,1.0,0.0,0.0,58.0,7.0,5.0,0,-1.0,-1.0,16.0,-1.0,0.0,-1.0,0.0,30
14998,144,144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,136.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [8]:
# 스케일링

In [9]:
# multiple_model 만 범주형 데이터

categorical_df = train[['multiple_model']]
numerical_df = train.drop(columns=['multiple_model'])
numerical_df

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,fwver_03.11.1141,fwver_03.11.1149,fwver_04.16.2641,fwver_04.16.3345,fwver_04.16.3439,fwver_04.16.3569,fwver_04.16.3571,fwver_04.22.1442,fwver_04.22.1656,fwver_04.22.1666,fwver_04.22.1684,fwver_04.22.1778,fwver_04.33.1095,fwver_04.33.1125,fwver_04.33.1149,fwver_04.33.1171,fwver_04.73.2571,fwver_04.82.1730,fwver_05.15.2090,fwver_05.15.2092,fwver_05.15.2114,fwver_05.15.2120,fwver_05.15.2122,fwver_05.15.3104,fwver_05.66.3571,fwver_8.5.3,errtype_1,errtype_2,errtype_3,errtype_5,errtype_6,errtype_8,errtype_9,errtype_10,errtype_13,errtype_14,errtype_15,errtype_17,errtype_18,errtype_19,errtype_20,errtype_22,errtype_24,errtype_25,errtype_26,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,quality_0,quality_1,quality_5,quality_6,quality_8,quality_11,quality_12,period
0,316,0.0,0.0,0.0,316.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,0.0,0.0,7.0,1.0,10.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,30
1,2345,0.0,0.0,2345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,1.0,0.0,0.0,0.0,1.0,8.0,143.0,0.0,4.0,1.0,3.0,756.0,5.0,1.0,22.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,0.0,0.0,113.0,56.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30
2,305,0.0,0.0,0.0,305.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,0.0,0.0,1.0,1.0,4.0,52.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,30.0,45.0,0.0,-1.0,0.0,29
3,297,0.0,0.0,297.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,45.0,1.0,0.0,0.0,0.0,28.0,0.0,0.0,57.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,2.0,0.0,17.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30
4,777,777.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,132.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,5.0,143.0,0.0,0.0,0.0,0.0,140.0,0.0,0.0,33.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,2.0,-1.0,-1.0,3.0,86.0,0.0,-1.0,0.0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,191,0.0,0.0,191.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,0.0,0.0,0.0,5.0,8.0,26.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,22.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0,-2.0,-2.0,3.0,-2.0,0.0,-2.0,0.0,10
14996,4,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
14997,818,818.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,358.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,1.0,0.0,0.0,0.0,3.0,1.0,181.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,48.0,0.0,223.0,0.0,16.0,12.0,0.0,1.0,0.0,0.0,58.0,7.0,5.0,-1.0,-1.0,16.0,-1.0,0.0,-1.0,0.0,30
14998,144,144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,136.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [10]:
# 수치형 데이터 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scalied_df = numerical_df.copy()

scalied_df = scaler.fit_transform(scalied_df)
numerical_df = pd.DataFrame(scalied_df, columns=numerical_df.columns)
numerical_df.head()

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,fwver_03.11.1141,fwver_03.11.1149,fwver_04.16.2641,fwver_04.16.3345,fwver_04.16.3439,fwver_04.16.3569,fwver_04.16.3571,fwver_04.22.1442,fwver_04.22.1656,fwver_04.22.1666,fwver_04.22.1684,fwver_04.22.1778,fwver_04.33.1095,fwver_04.33.1125,fwver_04.33.1149,fwver_04.33.1171,fwver_04.73.2571,fwver_04.82.1730,fwver_05.15.2090,fwver_05.15.2092,fwver_05.15.2114,fwver_05.15.2120,fwver_05.15.2122,fwver_05.15.3104,fwver_05.66.3571,fwver_8.5.3,errtype_1,errtype_2,errtype_3,errtype_5,errtype_6,errtype_8,errtype_9,errtype_10,errtype_13,errtype_14,errtype_15,errtype_17,errtype_18,errtype_19,errtype_20,errtype_22,errtype_24,errtype_25,errtype_26,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,quality_0,quality_1,quality_5,quality_6,quality_8,quality_11,quality_12,period
0,-0.268744,-0.340504,-0.239957,-0.414143,0.102027,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.277099,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,0.085244,-0.296006,-0.075511,-0.052361,-0.037669,0.025586,0.015363,-0.026017,-0.652306,-0.279724,-0.225162,-0.114028,-0.252677,-0.225553,-0.071956,-0.075181,-0.149787,-0.154519,-0.382093,-0.198956,-1.052305,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,-0.023049,0.120567,-0.059816,-0.058683,-0.044469,0.226409,-0.06688,0.442781
1,0.500848,-0.340504,-0.239957,4.011349,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.277099,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,0.042137,-0.075511,-0.052361,-0.037669,-0.048693,0.015363,-0.034766,0.560808,-0.279724,4.832761,2.461897,5.213885,1.147012,0.066194,0.877285,-0.590355,-0.154519,0.339732,-0.198956,-0.166879,0.083164,-0.074159,0.628662,-0.114398,-0.080131,0.795861,2.510621,-0.362551,-0.023049,0.120567,-0.059993,-0.058683,-0.044469,0.226409,-0.06688,0.442781
2,-0.272916,-0.340504,-0.239957,-0.414143,0.096218,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.277099,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.000473,-0.288962,-0.043544,-0.052361,-0.037669,-0.038082,0.015363,-0.052263,-0.753398,-0.279724,-0.225162,-0.114028,-0.252677,-0.225553,-0.016696,-0.075181,-0.458185,-0.154519,-0.382093,-0.198956,-1.052305,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,-0.022932,0.023923,-0.059551,-0.011320,-0.044469,0.091401,-0.06688,0.272376
3,-0.275951,-0.340504,-0.239957,0.146356,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.277099,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,-0.281917,-0.075511,-0.052361,-0.037669,-0.048693,-0.064734,-0.069761,-0.854491,-0.188357,-0.225162,-0.114028,-0.252677,-0.174717,-0.071956,-0.075181,0.951632,-0.154519,-0.194419,-0.198956,-0.343964,-0.061546,-0.074159,0.628662,0.641705,-0.080131,-0.392093,-0.285994,-0.644371,-0.023049,0.120567,-0.059993,-0.058683,-0.044469,0.226409,-0.06688,0.442781
4,-0.093888,0.651704,-0.239957,

In [11]:
train = numerical_df.join(categorical_df)
train.head()

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,fwver_03.11.1141,fwver_03.11.1149,fwver_04.16.2641,fwver_04.16.3345,fwver_04.16.3439,fwver_04.16.3569,fwver_04.16.3571,fwver_04.22.1442,fwver_04.22.1656,fwver_04.22.1666,fwver_04.22.1684,fwver_04.22.1778,fwver_04.33.1095,fwver_04.33.1125,fwver_04.33.1149,fwver_04.33.1171,fwver_04.73.2571,fwver_04.82.1730,fwver_05.15.2090,fwver_05.15.2092,fwver_05.15.2114,fwver_05.15.2120,fwver_05.15.2122,fwver_05.15.3104,fwver_05.66.3571,fwver_8.5.3,errtype_1,errtype_2,errtype_3,errtype_5,errtype_6,errtype_8,errtype_9,errtype_10,errtype_13,errtype_14,errtype_15,errtype_17,errtype_18,errtype_19,errtype_20,errtype_22,errtype_24,errtype_25,errtype_26,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,quality_0,quality_1,quality_5,quality_6,quality_8,quality_11,quality_12,period,multiple_model
0,-0.268744,-0.340504,-0.239957,-0.414143,0.102027,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.277099,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,0.085244,-0.296006,-0.075511,-0.052361,-0.037669,0.025586,0.015363,-0.026017,-0.652306,-0.279724,-0.225162,-0.114028,-0.252677,-0.225553,-0.071956,-0.075181,-0.149787,-0.154519,-0.382093,-0.198956,-1.052305,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,-0.023049,0.120567,-0.059816,-0.058683,-0.044469,0.226409,-0.06688,0.442781,0
1,0.500848,-0.340504,-0.239957,4.011349,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.277099,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,0.042137,-0.075511,-0.052361,-0.037669,-0.048693,0.015363,-0.034766,0.560808,-0.279724,4.832761,2.461897,5.213885,1.147012,0.066194,0.877285,-0.590355,-0.154519,0.339732,-0.198956,-0.166879,0.083164,-0.074159,0.628662,-0.114398,-0.080131,0.795861,2.510621,-0.362551,-0.023049,0.120567,-0.059993,-0.058683,-0.044469,0.226409,-0.06688,0.442781,0
2,-0.272916,-0.340504,-0.239957,-0.414143,0.096218,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.277099,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.000473,-0.288962,-0.043544,-0.052361,-0.037669,-0.038082,0.015363,-0.052263,-0.753398,-0.279724,-0.225162,-0.114028,-0.252677,-0.225553,-0.016696,-0.075181,-0.458185,-0.154519,-0.382093,-0.198956,-1.052305,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,-0.022932,0.023923,-0.059551,-0.011320,-0.044469,0.091401,-0.06688,0.272376,0
3,-0.275951,-0.340504,-0.239957,0.146356,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.277099,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,-0.281917,-0.075511,-0.052361,-0.037669,-0.048693,-0.064734,-0.069761,-0.854491,-0.188357,-0.225162,-0.114028,-0.252677,-0.174717,-0.071956,-0.075181,0.951632,-0.154519,-0.194419,-0.198956,-0.343964,-0.061546,-0.074159,0.628662,0.641705,-0.080131,-0.392093,-0.285994,-0.644371,-0.023049,0.120567,-0.059993,-0.058683,-0.044469,0.226409,-0.06688,0.442781,0
4,-0.093

In [12]:
train = train.to_numpy()
problem = np.zeros(15000)
problem[train_problem.user_id.unique() - 10000] = 1

train_x = train
train_y = problem

In [13]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):
    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 3
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Info] Number of positive: 3633, number of negative: 8367
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001612 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5519
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 54
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302750 -> initscore=-0.834237
[LightGBM] [Info] Start training from score -0.834237
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[2]	valid_0's auc: 0.795731	valid_0's pr_auc: 0.803296
[LightGBM] [Info] Number of positive: 4828, number of negative: 7172
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002235 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5709
[LightGBM] [Info] Numbe

In [14]:
print(np.mean(auc_scores))

0.7999523249712173


In [15]:
train_x = pd.DataFrame(train_x)
train_y = pd.DataFrame(train_y)
train_y.columns = ['problem']
train = pd.concat([train_x,train_y], axis =1)
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,problem
0,-0.268744,-0.340504,-0.239957,-0.414143,0.102027,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.277099,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,0.085244,-0.296006,-0.075511,-0.052361,-0.037669,0.025586,0.015363,-0.026017,-0.652306,-0.279724,-0.225162,-0.114028,-0.252677,-0.225553,-0.071956,-0.075181,-0.149787,-0.154519,-0.382093,-0.198956,-1.052305,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,-0.023049,0.120567,-0.059816,-0.058683,-0.044469,0.226409,-0.06688,0.442781,0.0,0.0
1,0.500848,-0.340504,-0.239957,4.011349,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.277099,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,0.042137,-0.075511,-0.052361,-0.037669,-0.048693,0.015363,-0.034766,0.560808,-0.279724,4.832761,2.461897,5.213885,1.147012,0.066194,0.877285,-0.590355,-0.154519,0.339732,-0.198956,-0.166879,0.083164,-0.074159,0.628662,-0.114398,-0.080131,0.795861,2.510621,-0.362551,-0.023049,0.120567,-0.059993,-0.058683,-0.044469,0.226409,-0.06688,0.442781,0.0,1.0
2,-0.272916,-0.340504,-0.239957,-0.414143,0.096218,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.277099,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.000473,-0.288962,-0.043544,-0.052361,-0.037669,-0.038082,0.015363,-0.052263,-0.753398,-0.279724,-0.225162,-0.114028,-0.252677,-0.225553,-0.016696,-0.075181,-0.458185,-0.154519,-0.382093,-0.198956,-1.052305,-0.061546,-0.074159,-1.030661,-0.114398,-0.080131,-0.602460,-0.336841,-0.644371,-0.022932,0.023923,-0.059551,-0.011320,-0.044469,0.091401,-0.06688,0.272376,0.0,0.0
3,-0.275951,-0.340504,-0.239957,0.146356,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,-0.277099,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,-0.281917,-0.075511,-0.052361,-0.037669,-0.048693,-0.064734,-0.069761,-0.854491,-0.188357,-0.225162,-0.114028,-0.252677,-0.174717,-0.071956,-0.075181,0.951632,-0.154519,-0.194419,-0.198956,-0.343964,-0.061546,-0.074159,0.628662,0.641705,-0.080131,-0.392093,-0.285994,-0.644371,-0.023049,0.120567,-0.059993,-0.058683,-0.044469,0.226409,-0.06688,0.442781,0.0,0.0
4,-0.093888,0.651704,-0.239957,-0.414143,-0.064861,-0.055185,-0.047764,-0.024537,-0.050453,-0.026486,-0.009173,-0.013527,-0.008165,-0.009359,-0.011326,-0.00921,-0.084309,-0.008948,-0.008165,-0.009902,-0.035379,0.164038,-0.010558,-0.01059,-0.019935,-0.012796,-0.026621,-0.008165,-0.008165,-0.008165,-0.008165,-0.016768,-0.008165,-0.010397,-0.026337,-0.014996,-0.196852,-0.030118,-0.029046,-0.296006,-0.011578,-0.052361,-0.037669,-0.048693,0.175556,-0.047889,0.560808,-0.279724,-0.225162,-0.114028,-0.252677,0.028626,-0.071956,-0.075181,-0.105730,-0.154519,0.126072,-0.198956,0.364376,-0.061546,-0.074159,0.628662,-0.114398,-0.080131,-0.552962,-0.336841,-0.080732,-0.023167,0.023923,-0.059949,0.031832,-0.044469,0.091401,-0.06688,0.442781,0.0,1.0
...,...,...,...,...,...,...,...,...,..

In [16]:
clf = setup(data = train, target = "problem") 

,Description,Value
0,session_id,6187
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 77)"
5,Missing Values,False
6,Numeric Features,68
7,Categorical Features,8
8,Ordinal Features,False
9,High Cardinality Features,False


In [17]:
best_3 = compare_models(sort = 'AUC', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7907,0.8068,0.5110,0.7843,0.6186,0.4832,0.5044,0.0730
gbc,Gradient Boosting Classifier,0.7945,0.8065,0.4838,0.8264,0.6099,0.4830,0.5152,0.3070
rf,Random Forest Classifier,0.7892,0.8035,0.4918,0.7969,0.6078,0.4746,0.5007,0.2010
et,Extra Trees Classifier,0.7863,0.7990,0.4904,0.7865,0.6039,0.4681,0.4928,0.1850
ada,Ada Boost Classifier,0.7888,0.7984,0.4930,0.7944,0.6080,0.4741,0.4997,0.0840
lr,Logistic Regression,0.7798,0.7704,0.4251,0.8299,0.5616,0.4345,0.4781,0.3800
lda,Linear Discriminant Analysis,0.7640,0.7547,0.3588,0.8403,0.5022,0.3788,0.4380,0.0370
knn,K Neighbors Classifier,0.7467,0.7277,0.4351,0.6893,0.5331,0.3713,0.3900,0.3700
nb,Naive Bayes,0.3507,0.7139,0.9808,0.3365,0.5010,0.0122,0.0429,0.0160
dt,Decision Tree Classifier,0.6928,0.6576,0.5537,0.5374,0.5451,0.3134,0.3137,0.0330


In [18]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7881,0.8101,0.4842,0.7991,0.6030,0.4701,0.4975
1,0.8100,0.8171,0.5229,0.8469,0.6466,0.5264,0.5550
2,0.7900,0.7982,0.4785,0.8127,0.6023,0.4724,0.5029
3,0.7957,0.8096,0.5029,0.8106,0.6207,0.4912,0.5174
4,0.7885,0.8037,0.4835,0.8005,0.6029,0.4704,0.4982
Mean,0.7945,0.8077,0.4944,0.8139,0.6151,0.4861,0.5142
Std,0.0082,0.0064,0.0165,0.0173,0.0172,0.0216,0.0216


In [19]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7983,0.8311,0.5374,0.7953,0.6414,0.5083,0.5272
